In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import datetime
import psycopg2
from sqlalchemy import create_engine
import sqlalchemy
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
dropbox_dir = r'/Users/kendraziegler/Dropbox (Palmer Lab)/Palmer Lab/Shae Ziegler/Suzanne_Mitchell_U01'
df_fits = pd.read_csv(dropbox_dir + '/fits.csv')
df_locomotor = pd.read_csv(dropbox_dir + '/locomotor.csv')
df_wfu = pd.read_csv(dropbox_dir + '/wfu_master.csv')
df_fecal_boli = pd.read_csv(dropbox_dir + '/fecal_boli.csv', dtype = str)

In [3]:
df_locomotor

,rfid,cohort,cage_x,locomotor_t1_experiment,locomotor_t1_time,locomotor_t1_file_num,locomotor_t2_time,locomotor_t2_file_num,locomotor_t2_experiment,sample,...,locomotor_t1_rest_episode_count,locomotor_t1_movement_episode_count,locomotor_t1_vertical_activity_count,locomotor_t1_center_time_legacy,locomotor_t2_total_distance,locomotor_t2_rest_time,locomotor_t2_rest_episode_count,locomotor_t2_movement_episode_count,locomotor_t2_vertical_activity_count,locomotor_t2_center_time_legacy
0,933000320045751,C02,4,U01-t1-gp20,t1,gp20,t2,gp20,U01-t2-gp20,120,...,130,139,1667,221.18,3011.69,432.70,136.0,143.0,1168.0,63.77
1,933000320045756,C02,4,U01-t1-gp27,t1,gp27,t2,gp27,U01-t2-gp27,120,...,141,141,868,10.54,1872.80,504.49,148.0,152.0,1109.0,15.39
2,933000320045757,C02,4,U01-t1-gp9,t1,gp9,t2,gp9,U01-t2-gp9,120,...,131,139,1703,151.58,3831.46,333.54,118.0,126.0,1135.0,54.23
3,933000320045759,C02,4,U01-t1-gp26,t1,gp26,t2,gp26,U01-t2-gp26,120,...,148,157,1569,110.57,3588.23,411.87,142.0,151.0,1350.0,93.32
4,933000320045760,C02,2,U01-t1-gp20,t1,gp20,t2,gp20,U01-t2-gp20,120,...,127,134,1297,454.05,1508.50,527.21,162.0,164.0,994.0,539.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,933000320187371,C05,4,U01-t1-gp25,t1,gp25,t2,gp25,U01-t2-gp25,120,...,134,142,1452,199.39,1801.18,513.65,152.0,155.0,996.0,90.15
492,933000320187372,C05,1,U01-t1-gp20,t1,gp20,t2,gp20,U01-t2-gp20,120,...,110,119,1749,108.24,2689.15,368.69,153.0,159.0,1578.0,87.50
493,933000320187373,C05,4,U01-t1-gp20,t1,gp20,t2,gp20,U01-t2-gp20,120,...,84,80,1073,41.92,2107.73,486.34,168.0,175.0,921.0,47.62
494,933000320187374,C05,4,U01-t1-gp16,t1,gp16,t2,gp16,U01-t2-gp16,120,...,138,146,1375,105.87,2602.75,404.15,168.0,175.0,1269.0,60.48


In [4]:
def dd_fits(df_fits):
    df_fits = df_fits.add_prefix('dd_')
    df_fits.rename(columns={'dd_sex':'sex','dd_cohort':'cohort','dd_rfid':'rfid'},inplace=True)
    df_fits.drop(['sex'],axis = 1, inplace = True)

    return df_fits
df_fits = dd_fits(df_fits)
#list(df_fits.columns.values)
#df_fits['cohort'].value_counts()


In [5]:
#find all rfid that are in dd_fits or locomotor
rfid_fits = list(df_fits['rfid'])
rfid_locomotor = list(df_locomotor['rfid'])
wfu_needed = list(set(rfid_fits + rfid_locomotor))
#wfu_needed

In [6]:
def locomotor(df_locomotor):
    df_locomotor.drop(['sample'],axis = 1, inplace = True)
    return df_locomotor
        
        
df_locomotor = locomotor(df_locomotor)        
#list(df_locomotor.columns.values)
#df_locomotor['cohort'].value_counts()
#df_locomotor

In [7]:
def fecal_boli():
    df_fecal_boli.rename(columns = {'box':'fecal_box','date_1':'fecal_date_1','time_in_1':'fecal_time_in_1'
                                    ,'date_2':'fecal_date_2','time_in_2':'fecal_time_in_2',
                                    'change':'fecal_change'},inplace=True)
    df_fecal_boli.drop(['sex','last_5'],axis = 1, inplace = True)

    
fecal_boli()
#df_fecal_boli['cohort'].value_counts()
#list(df_fecal_boli.columns.values)


In [8]:
def wfu_columns():
    df_wfu.drop(['last_5'],axis = 1, inplace = True)

wfu_columns()

In [9]:
#Changes the int rfid and last_5 columns into strings
def reformat(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)
        df[col] = [val.replace('.0','') for val in df[col]]
        
reformat(df_fecal_boli, ["rfid"])
reformat(df_fits, ["rfid"])
reformat(df_locomotor, ["rfid"])
reformat(df_wfu, ["rfid","housingbox"])


In [10]:
len(wfu_needed)

500

In [15]:
df_fits


,cohort,rfid,dd_coat_color,dd_shipping_box,dd_housing_box,dd_order_in_box,dd_squad_num,dd_box_color,dd_opertant_box,dd_id_check,...,dd_indiff_8,dd_indiff_16,dd_indiff_24,dd_hyperbolic_k,dd_hyperbolic_lnk,dd_hyperbolic_b,dd_AUC_traditional,dd_AUC_normal,dd_quasi_H_k_beta,dd_quasi_H_s_delta
0,C02,933000320045751,brown_hood,5,40,2,4,blue,8,0,...,49.205815,22.286333,30.258215,0.152786,-1.878717,0.794732,0.319148,0.401579,0.815571,0.905189
1,C02,933000320045756,black,11,54,2,5,blue,12,0,...,62.431850,37.152422,31.230381,0.083817,-2.479118,0.640505,0.353315,0.551620,0.595001,0.952806
2,C02,933000320045757,brown_hood,11,18,2,2,blue,12,0,...,43.284175,42.058929,29.908862,0.249252,-1.389293,0.796148,0.312850,0.392955,0.409958,0.971604
3,C02,933000320045759,brown_hood,10,52,2,5,blue,8,0,...,69.614073,31.810227,26.974896,0.151994,-1.883917,1.068606,0.431175,0.403493,0.987392,0.921055
4,C02,933000320045760,brown,5,39,2,4,blue,6,0,...,90.602227,45.685779,31.550220,0.172621,-1.756656,1.229121,0.498878,0.405882,1.008066,0.931037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,C04,933000320047963,black_hood,1,12,2,1,red,8,0,...,59.189177,53.073730,NaN,0.071768,-2.634312,0.998539,NaN,NaN,2.027855,0.831496
391,C04,933000320047964,black_hood,6,34,1,3,red,3,0,...,84.682184,66.694060,36.386721,0.007513,-4.891068,0.523546,0.576590,1.101316,1.291869,0.928345
392,C04,933000320047968,brown_hood,1,27,1,3,blue,5,0,...,116.862215,58.987025,46.116756,0.155155,-1.863329,1.378207,0.606703,0.440212,1.037585,0.948342
393,C04,933000320047973,black,1,12,1,1,red,7,0,...,113.984993,64.902099,37.109793,0.087389,-2.437385,1.134337,0.616921,0.543860,1.102845,0.945492


In [11]:
df_fits_locomotor = pd.merge(df_wfu, df_locomotor, on=['rfid','cohort'],how = 'outer')
df_wfu_fecal = pd.merge(df_fits_locomotor, df_fecal_boli, on=['rfid','cohort'],how = 'outer')
df_all = pd.merge(df_wfu_fecal, df_fits, on=['rfid','cohort'],how = 'outer')

df_all_final = df_all.loc[df_all['rfid'].isin(wfu_needed)]
df_all_final
#df_all_final['cohort'].value_counts()


,cohort,sires,dames,labanimalid,accessid,sex,rfid,dob,dow,shipmentdate,...,dd_indiff_8,dd_indiff_16,dd_indiff_24,dd_hyperbolic_k,dd_hyperbolic_lnk,dd_hyperbolic_b,dd_AUC_traditional,dd_AUC_normal,dd_quasi_H_k_beta,dd_quasi_H_s_delta


In [16]:
df_all

,cohort,sires,dames,labanimalid,accessid,sex,rfid,dob,dow,shipmentdate,...,dd_indiff_8,dd_indiff_16,dd_indiff_24,dd_hyperbolic_k,dd_hyperbolic_lnk,dd_hyperbolic_b,dd_AUC_traditional,dd_AUC_normal,dd_quasi_H_k_beta,dd_quasi_H_s_delta
0,C01,72539_1,72595_5,TJ008,73472_2,F,933000320045906,2018-09-24,2018-10-15,2018-10-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C01,73358_1,73360_1,TJ027,73477_2,F,933000320045890,2018-09-25,2018-10-16,2018-10-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C01,72539_1,72595_5,TJ003,73472_8,M,933000320045902,2018-09-24,2018-10-15,2018-10-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C01,72794_1,72775_4,TJ020,73475_4,F,933000320045904,2018-09-25,2018-10-16,2018-10-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C01,72624_1,72596_4,TJ002,73471_5,F,933000320045908,2018-09-23,2018-10-15,2018-10-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,C06,NaN,NaN,NaN,NaN,NaN,933000320148178,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
948,C06,NaN,NaN,NaN,NaN,NaN,933000320148244,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
949,C06,NaN,NaN,NaN,NaN,NaN,933000320148227,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
950,C06,NaN,NaN,NaN,NaN,NaN,933000320148074,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#df_all['time'].value_counts()
list(df_all_final.columns.values)

['cohort',
 'sires',
 'dames',
 'labanimalid',
 'accessid',
 'sex',
 'rfid',
 'dob',
 'dow',
 'shipmentdate',
 'litternumber',
 'littersize',
 'coatcolor',
 'earpunch',
 'rack',
 'shipmentbox',
 'housingbox',
 'shipmentage',
 'weanage',
 'comments',
 'resolution',
 'cage_x',
 'locomotor_t1_experiment',
 'locomotor_t1_time',
 'locomotor_t1_file_num',
 'locomotor_t2_time',
 'locomotor_t2_file_num',
 'locomotor_t2_experiment',
 'locomotor_t1_total_distance',
 'locomotor_t1_rest_time',
 'locomotor_t1_rest_episode_count',
 'locomotor_t1_movement_episode_count',
 'locomotor_t1_vertical_activity_count',
 'locomotor_t1_center_time_legacy',
 'locomotor_t2_total_distance',
 'locomotor_t2_rest_time',
 'locomotor_t2_rest_episode_count',
 'locomotor_t2_movement_episode_count',
 'locomotor_t2_vertical_activity_count',
 'locomotor_t2_center_time_legacy',
 'fecal_box',
 'fecal_date_1',
 'fecal_time_in_1',
 'fecal_boli_1',
 'fecal_date_2',
 'fecal_time_in_2',
 'fecal_boli_2',
 'fecal_change',
 'dd_coat